This is where the metadata, feature space, and genres are read in and stored in the "songs" dictionary by `track_id`.

In [1]:
from pkg.pickclassifier import *
from pkg.graphs import *
from pkg.run import *
from pkg.wrangling import *

In [2]:
default_song_data = FrozenMap(read_data('data.csv'))
genres = FrozenMap(read_genres('data.csv'))
default_feature_names = tuple(next(iter(default_song_data.values()))['features'].keys())

In [3]:
# Sanity check
l, dl = 0, 0
for x in default_song_data.values():
    if x['label'] == 1: l += 1
    else: dl += 1
print(' likes:', l, '\n', 'dislikes:', dl)

 likes: 1919 
 dislikes: 3790


In [4]:
test_cluster_size(default_song_data, 10)

2 3123.3948886881553
3 2481.7166322571657
4 2152.0688833043805
5 1944.1595684967979
6 1824.527518902902
7 1711.463680347652
8 1612.5454452933564
9 1536.3092762289934
10 1474.2760278975545


In [5]:
clustered_song_data, songs_by_cluster = {}, {}
clustered_song_data, songs_by_cluster = get_kmeans_clusters(default_song_data, NUM_CLUSTERS)

In [6]:
if set(next(iter(default_song_data.values()))['features'].keys()) == set(next(iter(clustered_song_data.values()))['features'].keys()):
    raise ValueError('Default features messed up.')

In [7]:
default_training_data, default_validation_data, clustered_training_data, clustered_validation_data = get_experiment_split(default_song_data, clustered_song_data)
training_clusters = get_train_clusters(clustered_training_data, songs_by_cluster[NUM_CLUSTERS])

In [8]:
print('----\nUnclustered\n----')
active_unclustered_results = run_active_suite(default_song_data, default_training_data, default_validation_data, SUPPORTED_ALGS, AL_STRATS)

----
Unclustered
----
101
svc accs w/ random
64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	
101
svc accs w/ uncertainty
64.15	85.97	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	85.36	35.84	81.15	64.15	82.73	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64

In [9]:
print('----\nClustered\n----')
active_clustered_results = run_active_suite(default_song_data, clustered_training_data, clustered_validation_data, SUPPORTED_ALGS, AL_STRATS)

----
Clustered
----
101
svc accs w/ random
64.15	80.1	64.15	84.57	64.15	81.41	64.15	85.88	64.15	87.37	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	35.84	87.9	64.15	64.15	64.15	87.9	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	
101
svc accs w/ uncertainty
64.15	64.15	64.15	70.02	64.15	64.15	64.15	64.15	64.15	64.15	64.15	66.78	35.84	65.81	35.84	70.02	35.84	35.84	35.84	85.45	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	64.15	6

In [10]:
print('\n----\nClustered w/ Cluster Sampling\n----')
active_cluster_sampled_results = run_clusters_suite(default_song_data, clustered_training_data, clustered_validation_data, training_clusters)


----
Clustered w/ Cluster Sampling
----
sgd accs w/ random
79.22	66.52	65.11	55.47	50.3	78.17	84.22	66.08	66.08	85.88	85.88	85.88	84.22	84.22	84.22	39.61	73.44	84.22	84.66	58.28	85.53	58.63	85.88	85.88	64.32	82.38	
sgd accs w/ uncertainty
86.94	64.24	64.24	53.81	53.81	51.88	64.68	55.12	86.41	85.53	65.55	65.55	76.16	69.23	36.54	87.29	35.93	81.5	81.5	77.56	80.54	80.54	80.54	81.15	80.36	80.01	


In [19]:

# # default_training_data, default_validation_data, clustered_training_data, clustered_validation_data

# with open('plots/' + str(NUM_CLUSTERS) + '_benchmarks.txt', 'w') as file:
#     svc_bench_unclustered = run_benchmark('svc', default_training_data, default_validation_data)
#     svc_bench_clustered = run_benchmark('svc', clustered_training_data, clustered_validation_data)
#     s = 'svc accs:\t' + str(svc_bench_unclustered) + '\t' + str(svc_bench_clustered)
#     file.write(s)
#     print(s)
#     lsvc_bench_unclustered = run_benchmark('lsvc', default_training_data, default_validation_data)
#     lsvc_bench_clustered = run_benchmark('lsvc', clustered_training_data, clustered_validation_data)
#     s = 'lsvc accs:\t' + str(lsvc_bench_unclustered) + '\t' + str(lsvc_bench_clustered)
#     file.write(s)
#     print(s)
#     sgd_bench_unclustered = run_benchmark('sgd', default_training_data, default_validation_data)
#     sgd_bench_clustered = run_benchmark('sgd', clustered_training_data, clustered_validation_data)
#     s = 'sgd accs:\t' + str(sgd_bench_unclustered) + '\t' + str(sgd_bench_clustered)
#     file.write(s)
#     print(s)
# # print()
# # print(active_unclustered_results)
# # print(active_clustered_results)
# # print(active_cluster_sampled_results)

In [12]:
# print(active_unclustered_results.keys())
# print(active_clustered_results.keys())
# print(active_cluster_sampled_results.keys())

In [13]:
# result_list = [active_unclustered_results, active_clustered_results]
# label_list = ['Unclustered', 'Clustered']

# # def get_new_results(alg):
# #     new_results = {}
# #     for i in range(len(result_list)):
# #         new_results[label_list[i]] = {}
# #         for k in result_list[i][alg]: new_results[label_list[i]][k] = result_list[i][alg][k]
# #     return new_results

# svc_results = get_new_results('svc')
# lsvc_results = get_new_results('lsvc')
# sgd_results = get_new_results('sgd')

In [14]:
# # def get_plottable_results(data):
# #     y, labels = [], []
# #     #print(svc_results.keys())
# #     for k in data:
# #         #print(svc_results[k].keys())
# #         for l in data[k]:
# #             #print(type(svc_results[k][l]))
# #             y.append(data[k][l])
# #             labels.append(k + ' ' + (' Random' if l == 'random' else ' Uncertainty'))
# #     return y, labels

# svc_y, svc_l = get_plottable_results(svc_results)
# lsvc_y, lsvc_l = get_plottable_results(lsvc_results)
# sgd_y, sgd_l = get_plottable_results(sgd_results)

# svc_y_r, svc_l_r = svc_y[::2], svc_l[::2]
# svc_y_u, svc_l_u = svc_y[1::2], svc_l[1::2]
# # lsvc_y_r, lsvc_l_r = lsvc_y[::2], lsvc_l[::2]
# # lsvc_y_u, lsvc_l_u = lsvc_y[1::2], lsvc_l[1::2]
# sgd_y_r, sgd_l_r = sgd_y[::2], sgd_l[::2]
# sgd_y_u, sgd_l_u = sgd_y[1::2], sgd_l[1::2]

In [15]:
# import matplotlib.pyplot as plt
# import matplotlib.ticker as mtick

# def accuracy_plot(y, labels, title, benchmark, save=False):
    
#     x = [i + 5 for i in range(len(y[0]))]
 
#     for i in range(len(y)):
#         plt.plot(x, y[i], '--' if i % 2 == 0 else '-', label=labels[i])
#     plt.plot(x, [benchmark for i in range(len(y[0]))], '-', label='Benchmark', color='black')
#     plt.legend()
#     plt.title(title)
#     plt.xlabel('# Training Instances')
#     plt.ylabel('Accuracy')
#     plt.axis([5, 5+len(y[0]), 0, 1.0])
#     plt.axes().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
#     if save: plt.savefig('plots/' + str(NUM_CLUSTERS) + '_' + title.replace(' ', '_') + '.png')
#     plt.show()

# accuracy_plot(svc_y_r, svc_l_r, 'SVC Random', svc_bench_acc)
# accuracy_plot(svc_y_u, svc_l_u, 'SVC Uncertainty', svc_bench_acc)
# accuracy_plot(lsvc_y, lsvc_l, 'LinearSVC', lsvc_bench_acc)
# accuracy_plot(sgd_y_r, sgd_l_r, 'SGD Random', sgd_bench_acc)
# accuracy_plot(sgd_y_u, sgd_l_u, 'SGD Uncertainty', sgd_bench_acc)

# # def get_accuracy_plots():
# #     for alg in SUPPORTED_ALGS:
# #         accuracy_plot(basic_results[i][:2], basic_titles[i])
        
# # get_accuracy_plots()


In [16]:
# result_list = [active_cluster_sampled_results]
# label_list = ['Clustered']
# sgd_c_results = get_new_results('sgd')
# sgd_c_y, sgd_c_l = get_plottable_results(sgd_c_results)

# # def accuracy_plot_c(y, labels, title, benchmark, save=False):
    
# #     # x = [i * (50//NUM_CLUSTERS) + 5 for i in range(len(y[0]))]
# #     # x = [i * (50//4) + 5 for i in range(len(y[0]))]
# #     x = [i + 5 for i in range(len(y[0]))]
 
# #     for i in range(len(y)):
# #         plt.plot(x, y[i], '--' if i % 2 == 0 else '-', label=labels[i])
# #     plt.plot(x, [benchmark for i in range(len(y[0]))], '-', label='Benchmark', color='black')
# #     plt.legend()
# #     plt.title(title)
# #     plt.xlabel('# Training Instances')
# #     plt.ylabel('Accuracy')
# #     plt.axis([5, 5+len(y[0]), 0, 1.0])
# #     plt.axes().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
# #     if save: plt.savefig('plots/' + str(NUM_CLUSTERS) + '_' + title.replace(' ', '_') + '.png')
# #     plt.show()

# accuracy_plot_c(sgd_c_y, sgd_c_l, 'SGD Cluster Sampling', sgd_bench_acc)

In [17]:
# TODO
# Add benchmark accuracy to all plots
# Try some more cluster numbers
# write report